## Notebook on Google Colab

First thing to do is to move the dataset to colab. To do this, we install the kaggle cli


In [1]:
!pip install kaggle

  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/2a/cc/c6/e4c872c053261660031b4a3e35ac958725b284c6fde9ff05c8
Successfully built kaggle


We need to move a file 'kaggle.json' to colab

In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


Now, we can download the competition dataset using the kaggle command

In [0]:
!mkdir /content/.kaggle
!mv kaggle.json /content/.kaggle/kaggle.json
!kaggle competitions download -c lagos-ai-hackathon
!unzip /content/.kaggle/competitions/lagos-ai-hackathon/Food.zip -d /content/.kaggle/competitions/lagos-ai-hackathon/images

### import packages that are required ###
and get the train and test data

In [1]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import cv2
import PIL as Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

train_path = "/content/.kaggle/competitions/lagos-ai-hackathon/images/train"
test_path = "/content/.kaggle/competitions/lagos-ai-hackathon/images/test"

from os import listdir
from os.path import isfile, join
train_labels = pd.read_csv("/content/.kaggle/competitions/lagos-ai-hackathon/labels.csv")
train_image_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
test_image_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]


Using TensorFlow backend.


# Creating the dataset

In [2]:
X = []
y = []
X_test = []
size = (224,224)

for i in train_image_files:
  imagearray = mpimg.imread(str(train_path)+"/"+i)
  imagearray = cv2.resize(imagearray,size, interpolation = cv2.INTER_CUBIC)
  imagearray = imagearray.ravel()
  X.append(imagearray)
  
  label = int(i.split("_")[0])
  y.append(int(label))
  
X_train = np.array(X)
y_train = np.array(y)


test_id = []

for i in test_image_files:
  imagearray = mpimg.imread(str(test_path)+"/"+i)
  imagearray = cv2.resize(imagearray,size, interpolation = cv2.INTER_CUBIC)
  imagearray = imagearray.ravel()
  X_test.append(imagearray)
  
  label = int(i.split(".")[0])
  test_id.append(int(label))

test_id = np.array(test_id)
X_test = np.array(X_test)

print("Shape of X_test: {0}".format(X_test.shape))
print("Shape of test_id: {0}".format(test_id.shape))
print("Shape of X_train: {0}".format(X_train.shape))
print("Shape of y_train: {0}".format(y_train.shape))



Shape of X_test: (3347, 150528)
Shape of test_id: (3347,)
Shape of X_train: (13296, 150528)
Shape of y_train: (13296,)


define some helper functions

In [0]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [0]:
def to_one_hot(x,n):
    b = np.zeros((x.shape[0],n))
    b[np.arange(x.shape[0]), x] = 1

    return b.astype(int)


Perform train/test split of train data

In [9]:
num_test = int(0.1 * X_train.shape[0])
X_t = (X_train[:num_test]).reshape(-1,3,224,224); y_t = y_train[:num_test]
X_train1 = (X_train[num_test:]).reshape(-1,3,224,224); y_train1 = y_train[num_test:]

print("X_train shape: {0}".format(X_train1.shape))
print("y_train shape: {0}".format(y_train1.shape))

print("X_test shape: {0}".format(X_t.shape))
print("y_test shape: {0}".format(y_t.shape))

X_train shape: (11967, 3, 224, 224)
y_train shape: (11967,)
X_test shape: (1329, 3, 224, 224)
y_test shape: (1329,)


In [0]:
X_train = X_train.reshape(-1,3,224,224)

Normalize the data (this code caused the kernel to die)

In [0]:
mean_pixel = [103.939, 116.779, 123.68]

img = X_train1
img = img.astype(np.float32, copy=False)
for c in range(3):
  img[:, c, :, :] = img[:, c, :, :] - mean_pixel[c]
X_train1 = img


img = X_test
img = img.astype(np.float32, copy=False)
for c in range(3):
  img[:, c, :, :] = img[:, c, :, :] - mean_pixel[c]
X_test = img

In [0]:
y_train_hot = to_one_hot(y_train,11)
#y_test_hot = to_one_hot(y_t,11)

# Model VGG-16
implementation obtained from fast.ai repository and tweeked

In [0]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

# In case we are going to use the TensorFlow backend we need to explicitly set the Theano image ordering
from keras import backend as K
K.set_image_dim_ordering('th')


vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr


class Vgg16():
    
    def __init__(self):
        self.FILE_PATH = 'http://files.fast.ai/models/'
        self.create()
        self.get_classes()


    def get_classes(self):
        
        fname = 'imagenet_class_index.json'
        fpath = get_file(fname, self.FILE_PATH+fname, cache_subdir='models')
        with open(fpath) as f:
            class_dict = json.load(f)
        self.classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

    def ConvBlock(self, layers, filters):
        """
            Adds a specified number of ZeroPadding and Covolution layers
            to the model, and a MaxPooling layer at the very end.

            Args:
                layers (int):   The number of zero padded convolution layers
                                to be added to the model.
                filters (int):  The number of convolution filters to be 
                                created for each layer.
        """
        model = self.model
        for i in range(layers):
            model.add(ZeroPadding2D((1, 1)))
            model.add(Convolution2D(filters, 3, 3, activation='relu'))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))


    def FCBlock(self):
        """
            Adds a fully connected layer of 4096 neurons to the model with a
            Dropout of 0.5

            Args:   None
            Returns:   None
        """
        model = self.model
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))


    def create(self):
        """
            Creates the VGG16 network achitecture and loads the pretrained weights.

            Args:   None
            Returns:   None
        """
        model = self.model = Sequential()
        model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))

        self.ConvBlock(2, 64)
        self.ConvBlock(2, 128)
        self.ConvBlock(3, 256)
        self.ConvBlock(3, 512)
        self.ConvBlock(3, 512)

        model.add(Flatten())
        self.FCBlock()
        self.FCBlock()
        model.add(Dense(1000, activation='softmax'))

        fname = 'vgg16.h5'
        model.load_weights(get_file(fname, self.FILE_PATH+fname, cache_subdir='models'))


    
    def ft(self, num):
        
        model = self.model
        model.pop()
        for layer in model.layers: layer.trainable=False
        model.add(Dense(1000, activation='relu'))
        model.add(Dense(500, activation='relu'))
        model.add(Dense(num, activation='softmax'))
        self.compile()

    def finetune(self, batches):
        
        self.ft(batches.nb_class)
        classes = list(iter(batches.class_indices)) # get a list of all the class labels
        
        # batches.class_indices is a dict with the class name as key and an index as value
        # eg. {'cats': 0, 'dogs': 1}

        # sort the class labels by index according to batches.class_indices and update model.classes
        for c in batches.class_indices:
            classes[batches.class_indices[c]] = c
        self.classes = classes


    def compile(self, lr=0.001):
        
        self.model.compile(optimizer=Adam(lr=lr),
                loss='categorical_crossentropy', metrics=['accuracy'])


    def fit_data(self, trn, labels,  val, val_labels,  nb_epoch=1, batch_size=64):
        
        self.model.fit(trn, labels, nb_epoch=nb_epoch,
                validation_data=(val, val_labels), batch_size=batch_size)
    def fit_val(self, X_train1, y_train_hot, batch_size=64, nb_epoch=3, verbose=2, validation_split=0.2):
        
        self.model.fit(X_train1, y_train_hot, batch_size=batch_size, nb_epoch=nb_epoch, verbose=verbose, validation_split=validation_split)

        
       

    def fit(self, batches, val_batches, nb_epoch=1):
        
        self.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=nb_epoch,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)


    def test(self, path, batch_size=8):
        
        test_batches = self.get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
        return test_batches, self.model.predict_generator(test_batches, test_batches.nb_sample)


# Compile and train

In [21]:
model = Vgg16() 

# fine tune the model by removing the last layer and adding a dense layer with 11 units
model.ft(11) 

# now, compile; using the default optimizer and learning rate
model.compile()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`


Train the model; (train for just 3 epochs)

In [0]:
model.fit_val(X_train, y_train_hot, batch_size=64, nb_epoch=3, verbose=2, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10636 samples, validate on 2660 samples
Epoch 1/3
 - 127s - loss: 2.2126 - acc: 0.2418 - val_loss: 2.0109 - val_acc: 0.2959
Epoch 2/3
 - 127s - loss: 2.0250 - acc: 0.2836 - val_loss: 1.9741 - val_acc: 0.3105
Epoch 3/3


Prredict on the test set

In [0]:
out = model.model.predict(X_test.reshape(-1,3,224,224))


In [14]:
out1 = np.argmax(out, axis = 1)
out1

array([10,  1,  2, ...,  9,  9,  5])

create submission dataframe

In [0]:
sub = pd.DataFrame({"image_id":test_id.tolist(),"label":out1.tolist()})

create submission csv file

In [0]:
sub.to_csv("/content/submission.csv", index=False)

download the submision.csv file from colab to local computer

In [0]:
from google.colab import files
files.download("submission.csv")